# read the csv

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import normalize
import spacy
from nltk.tokenize import word_tokenize
import re
from collections import Counter

In [2]:
df1 = pd.read_csv('../../csv/gpt/csv_gpt3-2.csv')
df2 = pd.read_csv('../../csv/human/csv_human-2.csv')

In [3]:
df1.to_csv('../csv/cfinal2.csv', index=False)
df2.to_csv('../csv/cfinal2.csv',mode='a',header=None,index=False) 

In [4]:
df3=pd.read_csv('../../csv/cfinal2.csv')

### remove unique as it is similar to lexical
### remove number number_of_repeatation, max_occurance_of_most_frequent_phrase

In [ ]:
# Get all columns of the DataFrame
columns_list = df3.columns

# Display the list of columns
print("Columns of the DataFrame:")
print(columns_list)

In [6]:
del df3['unique_words_rates']

In [7]:
del df3['number_of_repeatation']
del df3['max_occurance_of_most_frequent_phrase']

#### introducing stop words rates

In [8]:
def stop_w_r(str1):
    nw = word_tokenize(str1)
    n=len(nw)

    nlp = spacy.load("en_core_web_lg")
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop == True]
    #print(words)
    

    l=len(words)
    #print("n and l ",n,l)
    swr=l/n
    return swr

In [9]:
ess=df3.loc[:,'essay']
sw_list=[]

In [13]:
for i in range(len(ess)):
    swr=stop_w_r(ess[i])
    #print(swr)
    sw_list.append(swr)
    df3.at[i,'sw_rate']=swr

In [10]:
df3.insert(len(df3.columns) - 1, 'sw_rate', None)

In [11]:
columns_list = df3.columns

# Display the list of columns
print("Columns of the DataFrame:")
print(columns_list)

Columns of the DataFrame:
Index(['essay', 'topic', 'para_count', 'word_count', 'sentence_count',
       'sentence_len(mean)', 'sentence_len(max)', 'sentence_len(min)',
       'sw_rates', 'unique_words', 'noun', 'verb', 'adj', 'pron', 'modal verb',
       'occurrance_of_most_freq_word', 'rate_of_occurrance',
       'avg sentence complexity', 'lexical diversity', 'word_sim_p_count',
       'word_sim_p_score', 'word_sim_n_count', 'word_sim_n_score',
       'word_sim_score', 'flesch grade', 'Automated Readability',
       'coleman_liau', 'cohesion score', 'positive_sentiment',
       'negative_sentiment', 'neutral_sentiment', 'discourse_marker',
       'grammar', 'sw_rate', 'label'],
      dtype='object')


In [15]:
df3.to_csv('../csv/combined.csv', index=False)

n-gram repetation

In [2]:
df = pd.read_csv('../../csv/combined.csv')

In [4]:
new_column_names = ['sum_repeated_ngrams', 'count_repeated_ngrams', 'max_occurrence_ngrams']

# Get the index of the last column
last_column_index = df.columns.get_loc('label')

# Add three new columns with specific names before the last column with no values
for new_column_name in reversed(new_column_names):
    df.insert(last_column_index, new_column_name, None)

In [5]:
columns_list = df.columns

# Display the list of columns
print("Columns of the DataFrame:")
print(columns_list)

Columns of the DataFrame:
Index(['essay', 'topic', 'para_count', 'word_count', 'sentence_count',
       'sentence_len(mean)', 'sentence_len(max)', 'sentence_len(min)',
       'sw_rates', 'unique_words', 'noun', 'verb', 'adj', 'pron', 'modal verb',
       'occurrance_of_most_freq_word', 'rate_of_occurrance',
       'avg sentence complexity', 'lexical diversity', 'word_sim_p_count',
       'word_sim_p_score', 'word_sim_n_count', 'word_sim_n_score',
       'word_sim_score', 'flesch grade', 'Automated Readability',
       'coleman_liau', 'cohesion score', 'positive_sentiment',
       'negative_sentiment', 'neutral_sentiment', 'discourse_marker',
       'grammar', 'sw_rate', 'sum_repeated_ngrams', 'count_repeated_ngrams',
       'max_occurrence_ngrams', 'label'],
      dtype='object')


functions 

In [6]:
def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    list1=[" ".join(ngram) for ngram in ngrams]
    #return [" ".join(ngram) for ngram in ngrams]
    return list1

def rep_count(str1):
    list1=generate_ngrams(str1, 5)
    sentence_counter = Counter(list1)
    #print(sentence_counter)
    total_freq=0
    total_rep=0
    for sentence, frequency in sentence_counter.items():
        if frequency > 1:
            total_freq+=frequency
            total_rep+=1
            #print(f"Sentence: '{sentence}' - Frequency: {frequency}")
    #print("number of sentences appeared more than once ",total_rep," and total number of frequency(that are>1) ",total_freq)
    return total_freq,total_rep, sentence_counter.most_common(1)[0][1]

In [8]:
ess=df.loc[:,'essay']

In [ ]:
for i in range(len(ess)):
    print("################################ for essay number ",i," ###############################################")
    tf1,tr1,max_rep1=rep_count(ess[i])
    df.at[i,'sum_repeated_ngrams']=tf1
    df.at[i,'count_repeated_ngrams']=tr1 
    df.at[i,'max_occurrence_ngrams']=max_rep1

In [12]:
df.to_csv('../csv/combined_1.csv', index=False)

renaming sw_rates to avg_sw : avg_sw is stopwords per sentence and sw_rate(which is another column)

In [2]:
df = pd.read_csv('../../csv/combined_1.csv')

In [4]:
df = df.rename(columns={'sw_rates': 'avg_stop_word'})

In [5]:
columns_list = df.columns

# Display the list of columns
print("Columns of the DataFrame:")
print(columns_list)

Columns of the DataFrame:
Index(['essay', 'topic', 'para_count', 'word_count', 'sentence_count',
       'sentence_len(mean)', 'sentence_len(max)', 'sentence_len(min)',
       'avg_stop_word', 'unique_words', 'noun', 'verb', 'adj', 'pron',
       'modal verb', 'occurrance_of_most_freq_word', 'rate_of_occurrance',
       'avg sentence complexity', 'lexical diversity', 'word_sim_p_count',
       'word_sim_p_score', 'word_sim_n_count', 'word_sim_n_score',
       'word_sim_score', 'flesch grade', 'Automated Readability',
       'coleman_liau', 'cohesion score', 'positive_sentiment',
       'negative_sentiment', 'neutral_sentiment', 'discourse_marker',
       'grammar', 'sw_rate', 'sum_repeated_ngrams', 'count_repeated_ngrams',
       'max_occurrence_ngrams', 'label'],
      dtype='object')


# Eda

In [ ]:
import seaborn as sns
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

# Splitting the combined dataframe to human and chatgpt

In [ ]:
# Split the DataFrame based on the 'label' column
df1 = df[df['label'] == 1]
df2 = df[df['label'] == 0]

# Display the resulting DataFrames
print("DataFrame 1 (label=1):")
print(df1)

print("\nDataFrame 2 (label=0):")
print(df2)

In [ ]:
df2

In [ ]:
sns.pairplot(df)

In [ ]:
# Concatenate the DataFrames
combined_data = pd.concat([df1, df2])

# Create a pairplot with different colors for each DataFrame
sns.pairplot(combined_data, hue='label', markers=["o", "s"], palette={0: "b", 1: "r"})

# Show the plot
plt.show()

In [11]:
df.to_csv('../csv/combined_2.csv', index=False)

In [13]:
df = pd.read_csv('../../csv/combined_2.csv')

In [ ]:
# Select the columns to plot
x1 = df1["pron"]
y1 = df1["sw_rate"]

x2 = df2["pron"]
y2 = df2["sw_rate"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("pronun")
plt.ylabel("rate of stop words")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df1["coleman_liau"]
y1 = df1["sw_rate"]

x2 = df2["coleman_liau"]
y2 = df2["sw_rate"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("coleman_liau")
plt.ylabel("rate of stop words")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df1["lexical diversity"]
y1 = df1["sw_rate"]

x2 = df2["lexical diversity"]
y2 = df2["sw_rate"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("lexical diversity")
plt.ylabel("rate of stop words")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df1["noun"]
y1 = df1["sw_rate"]

x2 = df2["noun"]
y2 = df2["sw_rate"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("rate of noun")
plt.ylabel("rate of stop words")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df1["avg_stop_word"]
y1 = df1["flesch grade"]

x2 = df2["avg_stop_word"]
y2 = df2["flesch grade"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("avg sentence complexity")
plt.ylabel("flesch grade")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df2["avg sentence complexity"]
y1 = df2["flesch grade"]
# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Add labels and a legend
plt.xlabel("avg sentence complexity")
plt.ylabel("flesch grade")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df1["avg_stop_word"]
y1 = df1["Automated Readability"]

x2 = df2["avg_stop_word"]
y2 = df2["Automated Readability"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("avg sentence complexity")
plt.ylabel("Automated Readability")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [ ]:
# Select the columns to plot
x1 = df1["avg sentence complexity"]
y1 = df1["count_repeated_ngrams"]

x2 = df2["avg sentence complexity"]
y2 = df2["count_repeated_ngrams"]



# Plot the scatter plot for the first dataframe
plt.scatter(x1, y1, label="GPT", marker="o", c='skyblue',alpha=0.5)

# Plot the scatter plot for the second dataframe
plt.scatter(x2, y2, label="Human", marker="x",c='red', alpha=0.3)

# Add labels and a legend
plt.xlabel("average sentence complexity")
plt.ylabel("repetition of ngrams")
plt.legend()

# Show the plot
plt.title("Scatter Plot Comparison")
plt.grid()
plt.show()

In [3]:
df=pd.read_csv('../../csv/combined_2.csv')

In [5]:
columns_list = df.columns
print("Columns of the DataFrame:")
print(columns_list)

Columns of the DataFrame:
Index(['essay', 'topic', 'para_count', 'word_count', 'sentence_count',
       'sentence_len(mean)', 'sentence_len(max)', 'sentence_len(min)',
       'avg_stop_word', 'unique_words', 'noun', 'verb', 'adj', 'pron',
       'modal verb', 'occurrance_of_most_freq_word', 'rate_of_occurrance',
       'avg sentence complexity', 'lexical diversity', 'word_sim_p_count',
       'word_sim_p_score', 'word_sim_n_count', 'word_sim_n_score',
       'word_sim_score', 'flesch grade', 'Automated Readability',
       'coleman_liau', 'positive_sentiment', 'negative_sentiment',
       'neutral_sentiment', 'discourse_marker', 'grammar', 'sw_rate',
       'sum_repeated_ngrams', 'count_repeated_ngrams', 'max_occurrence_ngrams',
       'label'],
      dtype='object')


In [4]:
del df['cohesion score']

In [6]:
df.to_csv('../../csv/combined_3.csv', index=False)

In [ ]:
import seaborn as sns
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
df1.describe(include='all')

In [50]:
df1_description = df1.describe(include='all')
df1_description.to_csv('../xl/df1.csv', index=False)

In [51]:
excel_path = '../xl/df1.xlsx'
df1_description.to_excel(excel_path, index=False)

In [52]:
df2_description = df2.describe(include='all')
df2_description.to_csv('../xl/df2.csv', index=False)
excel_path = '../xl/df2.xlsx'
df2_description.to_excel(excel_path, index=False)

In [ ]:
df2.describe(include='all').T